# Feature Engineering with MKYZ

Learn how to create powerful features that improve model performance.

**Topics covered:**
- Polynomial features
- Interaction features
- DateTime features
- Lag features (time series)
- Rolling features
- Aggregation features
- Feature selection

In [1]:
import mkyz

mkyz package initialized. Version: 0.2.1


## 1. Load Sample Data

In [2]:
# Load the Titanic dataset
df = mkyz.load_data('data/titanic.csv')
print(f"Loaded {len(df)} rows with columns: {list(df.columns)}")
df.head()

Loaded 891 rows with columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Initialize FeatureEngineer

In [3]:
# Initialize the feature engineer
fe = mkyz.FeatureEngineer(random_state=42)
print("FeatureEngineer initialized!")

FeatureEngineer initialized!


## 3. Polynomial Features

Create non-linear relationships between numerical features.

In [4]:
# Create polynomial features for numerical columns
numerical_cols = ['Age', 'Fare']

# First, let's handle missing values in Age
df_poly = df.copy()
df_poly['Age'] = df_poly['Age'].fillna(df_poly['Age'].median())

# Create polynomial features of degree 2
df_poly = fe.create_polynomial_features(
    df_poly, 
    columns=['Age', 'Fare'], 
    degree=2
)

# Show new columns
new_cols = [col for col in df_poly.columns if 'poly' in col.lower() or '^' in col or '*' in col]
print(f"New polynomial features: {[col for col in df_poly.columns if col not in df.columns]}")

New polynomial features: ['Age^2', 'Age Fare', 'Fare^2']


## 4. Interaction Features

Capture relationships between multiple features.

In [5]:
# Create interaction features
df_interact = df.copy()
df_interact['Age'] = df_interact['Age'].fillna(df_interact['Age'].median())

df_interact = fe.create_interaction_features(
    df_interact,
    columns=['SibSp', 'Parch'],
    operations=['add', 'multiply']
)

# Show the new features
print("Created interaction features:")
interaction_cols = [col for col in df_interact.columns if col not in df.columns]
print(interaction_cols)
df_interact[interaction_cols].head()

Created interaction features:
['SibSp_x_Parch', 'SibSp_+_Parch']


,SibSp_x_Parch,SibSp_+_Parch
0,0,1
1,0,1
2,0,0
3,0,1
4,0,0


## 5. Feature Selection

Select the most important features for modeling.

In [6]:
# Prepare data for feature selection
data = mkyz.prepare_data(
    'data/titanic.csv',
    target_column='Survived',
    test_size=0.2,
    random_state=42
)

X_train, X_test, y_train, y_test, df_processed, target, num_cols, cat_cols = data

print(f"X_train shape: {X_train.shape}")
print(f"Number of features: {X_train.shape[1]}")

INFO:mkyz.data_processing:First 5 rows of the dataset:
INFO:mkyz.data_processing:   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803 

X_train shape: (576, 1420)
Number of features: 1420


In [7]:
# Select top 10 features using mutual information
selected_features = mkyz.select_features(
    X_train, 
    y_train, 
    k=10, 
    method='mutual_info',
    task='classification'
)

print(f"\nTop 10 selected features:")
for i, feat in enumerate(selected_features, 1):
    print(f"  {i}. {feat}")


Top 10 selected features:
  1. feature_2
  2. feature_15
  3. feature_86
  4. feature_724
  5. feature_725
  6. feature_892
  7. feature_914
  8. feature_1112
  9. feature_1290
  10. feature_1406


## 6. Train Model with Selected Features

In [8]:
# Use only selected features
# Since prepare_data returns numpy arrays, but select_features returns string names (e.g., 'feature_0'),
# we need to extract the indices to slice the arrays.
selected_indices = [int(f.split('_')[1]) for f in selected_features]

X_train_selected = X_train[:, selected_indices]
X_test_selected = X_test[:, selected_indices]

# Create new data tuple with selected features
data_selected = (X_train_selected, X_test_selected, y_train, y_test, df_processed, target, num_cols, cat_cols)

# Train a model with selected features
model = mkyz.train(
    data_selected,
    task='classification',
    model='rf',
    n_estimators=100,
    random_state=42
)

print("Model trained successfully!")

Model trained successfully!


In [9]:
# Evaluate the model
predictions = model.predict(X_test_selected)
metrics = mkyz.classification_metrics(y_test, predictions)

print("\nModel Performance with Selected Features:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")


Model Performance with Selected Features:
  accuracy: 0.7655
  precision: 0.7655
  recall: 0.7655
  f1_score: 0.7655
  mcc: 0.4811
  cohen_kappa: 0.4811


## 7. Track Created Features

The FeatureEngineer keeps track of all created features.

In [10]:
# View all features created during this session
print(f"Total features created: {len(fe.created_features)}")
print("\nCreated features:")
for feat in fe.created_features:
    print(f"  - {feat}")

Total features created: 5

Created features:
  - Age^2
  - Age Fare
  - Fare^2
  - SibSp_x_Parch
  - SibSp_+_Parch


## Summary

In this notebook, we learned:

1. **Polynomial Features** - Create non-linear relationships
2. **Interaction Features** - Combine features using mathematical operations
3. **Feature Selection** - Select the most important features
4. **Model Training** - Train models with engineered features

### Next Steps

- Try different feature selection methods (`f_score`)
- Experiment with higher polynomial degrees
- Apply datetime features to time-based datasets